In [ ]:
from web3 import Web3
from datetime import datetime
import pandas as pd
import json

In [ ]:
infura_url= 'https://mainnet.infura.io/v3/5c619862a80846bbb6b6362f09485a2b'
web3= Web3(Web3.HTTPProvider (infura_url))
web3.is_connected()

# Get Address from transaction hex

In [ ]:
address= web3.to_checksum_address("0x1ad91ee08f21be3de0ba2ba6918e714da6b45836")
transaction= '0x032ed60363beb809a2b9c9790bb7dadd83b743040945a087aeecbe9e6b2dc2af'

# Get Nonce

## A nonce is a cumulative number that keeps track

of the total transactions sent by the executing address. Smart contracts also have a nonce that keeps track of the number of contract creations.

The Ethereum Yellow Paper defines it as follows: `nonce: A scalar value equal to the number of
transactions sent from this address or, in the case of accounts with associated code, the number of contract-creations made by this account.`

the nonce provides information about the sending address. It serves two purposes:

- establishes the order of transaction processing for each address.
A transaction with a higher nonce number will not be processed
until all transactions with smaller nonces have been validated.

- identifies a transaction. Multiple transactions with the same
nonce can be sent to mempool, but once a transaction with a
particular nonce is added to the blockchain, subsequent
transactions with the same nonce will not be processed.

In [ ]:
next_nonce= web3.eth.get_transaction_count(address)
next_nonce

In [ ]:
transaction_nonce= web3.eth.get_transaction(transaction).nonce
transaction_nonce

# Gas price (gasPrice, as seen in Figure 2.2) 

- is determined by the sender of the transaction and
represents the price they are willing to pay for
each unit of gas.

- Transactions with higher gas prices are prioritized for confirmation.

- The gas price is denoted in Wei per unit of gas. 

In [55]:
gasPrice= web3.eth.get_transaction(transaction).gasPrice
gasPrice

19209576968

# Gas limit

## Gas limit (or gas) represents the maximum number of gas units (money invested) that the sender is willing to spend on the transaction. It is the upper limit of gas consumption for a transaction.

In [ ]:
gaslimit= web3.eth.get_transaction(transaction).gas
gaslimit

In [ ]:
gasUsed= web3.eth.get_transaction_receipt (transaction).gasUsed
gasUsed

# recepient

Depending on what information this field has, we
can classify transactions into these categories:

- • Regular: A transaction from one address to another address.

- • Contract deployment: A transaction where the to field is null
and where the data field is used for the contract code. In this
case, the sender is creating a smart contract.
- • Execution of a contract: The to field is a smart contract address,
and therefore the transaction represents the interaction with a
deployed smart contract.

In [ ]:
recipient= web3.eth.get_transaction(transaction).to
recipient

In [ ]:
sender= web3.eth.get_transaction(transaction)["from"]
sender

## The value of the transaction is expressed in wei, which can be converted to ether by multiplying it by 10**-18 or with web3.fromWei('value','ether')

In [ ]:
value= web3.eth.get_transaction(transaction).value
value*10**-18

## Retrieve the input data in a given transaction using the web3.eth.get_transaction() method specifying the input property.

In [ ]:
data= web3.eth.get_transaction(transaction).input
data

In [ ]:
transaction_ba='0xb3827bb3cca1a693ec69edb744755f64d8ff8c90f89f69cbfbfafd17b0083159'

In [ ]:
data_ba= web3.eth.get_transaction(transaction_ba).input
data_ba

In [ ]:
bored_ape= web3.eth.get_transaction(transaction_ba)

In [ ]:
sc_address="0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D"
with open ("./ba_abi.json") as f:
    abi= json.load (f)


# There are alternative tools to retrieve the ABI when the code is not verified. These tools,although still in beta, are listed in the Further reading section.

In [ ]:
baContract = web3.eth.contract(address=sc_address, abi=abi)

In [56]:
function, parameters = baContract.decode_function_input(bored_ape["input"])
print (function, 'parameters: ',parameters )

<Function mintApe(uint256)> parameters:  {'numberOfTokens': 1}


### As we can see, the mintApe function was triggered by this transaction, and Ape 6633 was born.

### V,R,S
These are the components of the signature by the
from address. The digital signature gives validity
that the message sent was executed by the
originating address. `For more on this matter, you
can read the Mastering Ethereum book`. Depending
on where we obtain our transaction data, we may
not encounter this field (for example, Covalent
does not provide it when querying for a specific
transaction).

### Transaction receipt

Once the transaction is executed, we gain access
to a valuable source of information, called a
transaction receipt. This receipt stores
information that results from executing the
transaction.
Of particular interest to us are the following
fields: `status`, `gasUsed`, `cumulativeGasUsed`, and
`logs`.

### Gas used and Cumulative gas used
The Gas used field represents the amount of gas
that was actually consumed by the transaction.
This value will be below the gas limit. We can
retrieve it with the following code:

In [58]:
web3.eth.get_transaction_receipt(transaction).gasUsed

21000

Additionally, we have the Cumulative gas used
field, which provides the total amount of gas
consumed by all previous transactions within the
same block, including the transaction under
analysis.

Gas prices are crucial data points that analysts
often consider. These fields represent the actual
cost of using the blockchain for a specific
transaction. Analyzing gas costs can help answer
questions such as: What is the best time of the
day to execute a set of transactions? When should
we deploy a set of smart contracts to minimize
gas expenses?.... and so on.

# Logs

Any change on the blockchain can be stored in
state data or event logs. The latter is less
expensive and provides a very rich source of
information.

In the context of smart contract operations, logs
are short pieces of information that record
specific events at a given point in time.

An event log, or log record, can be created by
providing specific instructions to the EVM. These
instructions are known as opcodes, and there are
five opcodes that emit event logs. The Ethereum
Yellow Paper provides more information about
opcodes under the title Logging operations

Log events consist of two components: topics and
data. When examining the source code, we will
find that log events are preceded by the `event` or
`emit` keyword.

### Topics describe the nature of an event and can include up to a maximum of four topics per log.
Typically, topics consist of the event’s signature
or name, along with the type of parameters
passed.

Topics are searchable but limited in terms of the
number of bytes they can include. As a result, the
information contained within topics is usually
referential. In the context of a Python dictionary,
topics can be considered as keys.

### Data, the other component, has no such limit in
terms of number of bytes and is less expensive to
store on the blockchain. This allows for more
flexibility in the information that can be
included, such as larger strings. If we were to
compare this with a Python dictionary, data
would correspond to the values.

Let’s continue the examination of the minting of
Bored Ape 6633. Depending on the data provider
we use to query the blockchain, we may receive
the logs already decoded or we might need to
decode them ourselves. To be able to decode
them, we will need the ABI, as extracted in the
previously analyzed Input data section. The code
essentially compares events from the ABI with
events from the log.

If there is a match, the code
translates the log.

the entire
process, which is summarized in the following
steps:
1. Creates a list of events from the contract.
2. Generates a hexadecimal value for each event extracted from the
ABI.
3. Extracts the receipt for the transaction.
4. Iterates through each log and compares the hexadecimal value of
the first parameter of the log (the topic) with the list created in step
2. If there is a match, the code uses the name within the
`process_receipt` function, and it will return a tuple with the
decoded logs

# Decoding Events

## Create a list of events listed in the the object baContract. Loop through the abi_events list and create a dictionary called event_abi_hex. The dictionary will contain the event name as the key and the event ABI in hexadecimal format as the value.

In [ ]:
abi_events = [abi for abi in baContract.abi if abi["type"] == "event"]
event_abi_hex = {}
for event in abi_events:
    name = event["name"]
    inputs = ",".join([param["type"] for param in event["inputs"]])
    event_abi_human = f"{name}({inputs})"
    event_abi_hex[name] = web3.to_hex(web3.keccak(text=event_abi_human))

## Retrieve the transaction receipt for the transaction_ba and loop through the logs in the receipt. For each log, it will get the event in hexadecimal format from the first topic and then compare it to the event_abi_hex dictionary. If a match is found, it will decode the log using processReceipt(tx_receipt)

In [ ]:
tx_receipt = web3.eth.get_transaction_receipt(transaction_ba)
decoded_logs = []
for log in tx_receipt['logs']:
    event_hex = web3.to_hex(log['topics'][0])
    event_name = None
    for name, hex_value in event_abi_hex.items():
        if hex_value == event_hex:
            event_name = name
            break
    if event_name is not None:
        decoded_log = baContract.events[event_name]().process_receipt(tx_receipt)
        decoded_logs.append(decoded_log)
decoded_logs